In [6]:
# aspects: inhouse generation and purchased 

# inputs:
# inhouse generation = Wi(t)
# price of electricity = Ci(t)
# the total incoming load = L(t)
# the QoS parameter = q

# outpus:
# the allocated workload to each datacenter
# green workload, Ygi
# brown workload, Ybi
# total workload, Y


# Algorithm:
# i)    assume we have n datacenter
# ii)   assume we are optimizing for each datacenter at once, and time slot are divided in hours
# iii)  assume we have N division in the timeslot
# iv)   for each time slot we will calculate the Y

# Idea:
# i)    we will consider Wi(t) to be a constant or zero
# ii)   we will consider the price to be the announced per unit price of energy( electricity )
# iii)  incoming load can be the load of peek hour
# iv)   QoS parameter neglected


# Brown Energy Treatment and Termination Algorithm
# columns: 'time', 'total_incoming_green_energy', 'total_incoming_brown_energy',
    #    'total_incoming_load', 'total_actual_load', 'green_energy_price',
    #    'brown_energy_price'
class Better(object):
    def __init__(self, dataframe):
        # the replicated dataset
        self.df = dataframe.copy()

        # the columns in ndarray
        self.time = self.df['time'].values
        self.total_incoming_green_energy = self.df['total_incoming_green_energy'].values
        self.total_incoming_brown_energy = self.df['total_incoming_brown_energy'].values
        self.total_incoming_load = self.df['total_incoming_load'].values
        self.total_actual_load = self.df['total_actual_load'].values
        self.green_energy_price = self.df['green_energy_price'].values
        self.brown_energy_price = self.df['brown_energy_price'].values

        #iterators
        self.time_it = iter(self.time)
        self.total_incoming_green_energy_it = iter(self.total_incoming_green_energy)
        self.total_incoming_brown_energy_it = iter(self.total_incoming_brown_energy)
        self.total_incoming_load_it = iter(self.total_incoming_load)
        self.total_actual_load_it = iter(self.total_actual_load)
        self.green_energy_price_it = iter(self.green_energy_price)
        self.brown_energy_price_it = iter(self.brown_energy_price)
        
        # params
        self.batch_size = 24

    def lenth(self):
        return len(self.df)
    
    def __len__(self):
        return len(self.df)
    
    def days(self):
        return len(self.df)/self.batch_size
    
    def load_batch(self, batch_size, it):
        buffer = []
        for _ in range(batch_size):
            buffer.append(next(it))
        return buffer

    def evalute(self, days=None):
        if not days:
            days = self.days()
        
        for _ in range(days):
            # green workload, Ygi
            Ygi = self.load_batch(batch_size=self.batch_size, it=self.total_incoming_green_energy_it)
            # brown workload, Ybi
            Ybi = self.load_batch(batch_size=self.batch_size, it=self.total_incoming_green_energy_it)
            # actual load, L
            L = self.load_batch(batch_size=self.batch_size, it=self.total_actual_load_it)

            # per day savings
            SAVE = 0
            SAVE_G = 0
            SAVE_B = 0
            # total workload, Y (per day)
            Y = 0
            # for test
            L_cum = 0
            # per time slot
            for i, (Lgi, Lbi, Li) in enumerate(zip(Ygi, Ybi, L)):
                L_cum += Li
                # per hour savings
                save = 0
                save_g = 0
                save_b = 0
                # per hour workload
                yi = 0
                if Lgi<Li:
                    if Lbi>Li:
                        yi += Li
                        save_g += Li-Lgi
                    else:
                        yi += Lbi
                else:
                    if Lgi > Li:
                        yi += Li
                        save_b += Li-Lgi
                    else:
                        yi = Lgi
                save = save_g + save_b
                SAVE_G += save_g
                SAVE_B += save_b
                Y += yi
                print(f'Hour {i+1}: Green Energy Saved: {save_g:.2f}, Brown Energy Saved:{save_b:.2f}')
            SAVE += SAVE_G + SAVE_B
            print('')
            print(f'Total Day workload: {Y:.2f} , Total Energy Saved: {SAVE:.2f}')
            print('')
            print(f'Total Green Energy Saved: {SAVE_G:.2f} , Total Brown Energy Saved: {SAVE_B:.2f}')
            print('')
            if L_cum == Y:
                print('Verdict: Fully Utilized!!!')
            else:
                print(f'Verdict: Deficiency!!!\n\t{L_cum-Y:.2f} energy deficit!')
                
                


In [7]:
# test 
import pandas as pd

df = pd.read_csv('preprocessed.csv')

# om
b = Better(dataframe=df)

b.evalute(1)

Hour 1: Green Energy Saved: 5.12, Brown Energy Saved:0.00
Hour 2: Green Energy Saved: 4.91, Brown Energy Saved:0.00
Hour 3: Green Energy Saved: 4.11, Brown Energy Saved:0.00
Hour 4: Green Energy Saved: 3.46, Brown Energy Saved:0.00
Hour 5: Green Energy Saved: 2.79, Brown Energy Saved:0.00
Hour 6: Green Energy Saved: 2.57, Brown Energy Saved:0.00
Hour 7: Green Energy Saved: 2.72, Brown Energy Saved:0.00
Hour 8: Green Energy Saved: 3.18, Brown Energy Saved:0.00
Hour 9: Green Energy Saved: 3.14, Brown Energy Saved:0.00
Hour 10: Green Energy Saved: 3.12, Brown Energy Saved:0.00
Hour 11: Green Energy Saved: 3.40, Brown Energy Saved:0.00
Hour 12: Green Energy Saved: 4.20, Brown Energy Saved:0.00
Hour 13: Green Energy Saved: 4.40, Brown Energy Saved:0.00
Hour 14: Green Energy Saved: 4.69, Brown Energy Saved:0.00
Hour 15: Green Energy Saved: 4.85, Brown Energy Saved:0.00
Hour 16: Green Energy Saved: 4.15, Brown Energy Saved:0.00
Hour 17: Green Energy Saved: 4.55, Brown Energy Saved:0.00
Hour 1